### 00. 크롬 드라이버 열기 

In [7]:
# 전체화면 아닐 때 예외처리 완료
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time as time
from bs4 import BeautifulSoup as bs
import re

In [8]:
# 크롬드라이버 열기
driver = webdriver.Chrome()
driver.maximize_window() # 크롬창 크기 최대

# 드라이버가 해당 url 접속
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)


### 01. 차트 파인더 조건 클릭하기

In [9]:
#차트파인더 클릭
driver.find_element(By.CSS_SELECTOR, 'div.chart_finder').click()
time.sleep(2)

#월간차트 클릭
driver.find_element(By.CSS_SELECTOR, 'h4.tab02').click()
time.sleep(2)

# 연대선택 2020년 클릭
driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label').click()
time.sleep(2)

# 연대선택 2023년 클릭
# 규칙성이 있기 때문에 2024년은 li[1] 2023년은  li[2] 2022년은 li[3]으로 바꿔주면 됨.
driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label').click()
time.sleep(2)

# 일단 월간 탭을 활성화 시켜야 하므로 1월 클릭
driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label').click()



 ### 02. 클릭 버튼 활성화된 month까지 가져오도록  month_active_list 만들어주기

In [11]:
# 2023년은 12월까지 결과가 다 나왔기 때문에 month_active_list가 01월부터 12월까지 만들어진다. 
# 2023포함 이전 연도는 12월까지 결과가 다 나왔기 때문에  12가 출력되고, # 2024년은 2월까지만 출력된다

month_active_list = driver.find_element(By.CSS_SELECTOR, 'div.box_chic.nth3.view.on > div.list_value').text
active_last_month = int(month_active_list.split('\n')[-1].replace('월','').strip()) # 2023년 활성화된 마지막월은 12
active_last_month


12

### 03. 12월까지 반복수행 하여 빈 리스트에 지정한 값을 담는다

In [13]:

year_list = []
month_list = []
rank_list = []
title_list = []
artist_list = []
album_list = []

# 12월까지  12번 반복해서 월, 장르, 검색 버튼 클릭하고 연도와 월 가져오는 for문
for month in range(active_last_month):  
 
    #월 버튼 클릭 
    month_tab = f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{month+1}]/span/label' 
    driver.find_element(By.XPATH, month_tab).click()
    time.sleep(2)
    
    # 장르 종합 선택
    driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[3]/span/label').click()
    time.sleep(2)
    
    # 검색 버튼 클릭
    driver.find_element(By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span/span').click()
    time.sleep(2)
    
    #해당 월의 정보 파싱해오기 
    html = driver.page_source
    soup = bs(html, 'html.parser')
    song_soup = soup.select('tbody > tr')
    month_soup = soup.select('span.datelk')[1].get_text().replace('\n', '')
    year_soup = soup.select('span.datelk')[0].get_text().replace('\n', '')
    
    # 해당 월마다 rank, title, artist, album 정보 가져오는 for문 
    for song in song_soup:
        # list에 append하면서 쓸데없는 문자, 공백 replace를 통해 삭제
        rank = song.find('span', class_= 'rank').get_text().replace('\n', '')
        title = song.find('div', class_= 'ellipsis rank01').get_text().replace('\n', '')
        artist = song.find('a', class_= 'fc_mgray').get_text()
        album = song.find('div', class_= 'ellipsis rank03').get_text().replace('\n', '')
        rank_list.append(rank)
        title_list.append(title)
        artist_list.append(artist)
        album_list.append(album)
        year_list.append(year_soup)
        month_list.append(month_soup)
        
     

chart_df_23 = pd.DataFrame(columns=['year','month','rank','title','artist','album'],
                         data=zip(year_list,month_list,rank_list, title_list, artist_list, album_list))
    


In [14]:
chart_df_23.head(5)

,year,month,rank,title,artist,album
0,2023,01,1,Ditto,NewJeans,NewJeans 'OMG'
1,2023,01,2,OMG,NewJeans,NewJeans 'OMG'
2,2023,01,3,Hype Boy,NewJeans,NewJeans 1st EP 'New Jeans'
3,2023,01,4,사건의 지평선,윤하 (YOUNHA),YOUNHA 6th Album Repackage 'END THEORY : Final...
4,2023,01,5,ANTIFRAGILE,LE SSERAFIM (르세라핌),ANTIFRAGILE


In [16]:
# 100개씩 12개월이니 1200개 정상적으로 담김을 확인
chart_df_23.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    1200 non-null   object
 1   month   1200 non-null   object
 2   rank    1200 non-null   object
 3   title   1200 non-null   object
 4   artist  1200 non-null   object
 5   album   1200 non-null   object
dtypes: object(6)
memory usage: 56.4+ KB
